<a href="https://colab.research.google.com/github/JosiasRuiz/AdvanceML/blob/main/TC4033_Activity4_Group43.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## TC 5033
### Text Generation

<br>

#### Activity 4: Building a Simple LSTM Text Generator using WikiText-2
<br>

- Objective:
    - Gain a fundamental understanding of Long Short-Term Memory (LSTM) networks.
    - Develop hands-on experience with sequence data processing and text generation in PyTorch. Given the simplicity of the model, amount of data, and computer resources, the text you generate will not replace ChatGPT, and results must likely will not make a lot of sense. Its only purpose is academic and to understand the text generation using RNNs.
    - Enhance code comprehension and documentation skills by commenting on provided starter code.
    
<br>

- Instructions:
    - Code Understanding: Begin by thoroughly reading and understanding the code. Comment each section/block of the provided code to demonstrate your understanding. For this, you are encouraged to add cells with experiments to improve your understanding

    - Model Overview: The starter code includes an LSTM model setup for sequence data processing. Familiarize yourself with the model architecture and its components. Once you are familiar with the provided model, feel free to change the model to experiment.

    - Training Function: Implement a function to train the LSTM model on the WikiText-2 dataset. This function should feed the training data into the model and perform backpropagation.

    - Text Generation Function: Create a function that accepts starting text (seed text) and a specified total number of words to generate. The function should use the trained model to generate a continuation of the input text.

    - Code Commenting: Ensure that all the provided starter code is well-commented. Explain the purpose and functionality of each section, indicating your understanding.

    - Submission: Submit your Jupyter Notebook with all sections completed and commented. Include a markdown cell with the full names of all contributing team members at the beginning of the notebook.
    
<br>

- Evaluation Criteria:
    - Code Commenting (60%): The clarity, accuracy, and thoroughness of comments explaining the provided code. You are suggested to use markdown cells for your explanations.

    - Training Function Implementation (20%): The correct implementation of the training function, which should effectively train the model.

    - Text Generation Functionality (10%): A working function is provided in comments. You are free to use it as long as you make sure to uderstand it, you may as well improve it as you see fit. The minimum expected is to provide comments for the given function.

    - Conclusions (10%): Provide some final remarks specifying the differences you notice between this model and the one used  for classification tasks. Also comment on changes you made to the model, hyperparameters, and any other information you consider relevant. Also, please provide 3 examples of generated texts.



# Group 43

* Andrea M. Ruiz G. - A01794631
* Josías Ruiz P. - A00968460
* Saúl Y. Salgueiro L. - A0XXXXXXX
* Jesús Á. Salazar M. - A00513236

Instalación de la paqueteria necesaria.

In [2]:
!pip install 'portalocker>=2.0.0'

Importar librerias necesarias

In [3]:
import numpy as np
#PyTorch libraries
import torch
import torchtext
from torchtext.datasets import WikiText2
# Dataloader library
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data.dataset import random_split
# Libraries to prepare the data
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.functional import to_map_style_dataset
# neural layers
from torch import nn
from torch.nn import functional as F
import torch.optim as optim
from tqdm import tqdm

import random

Configuracion para usar GPU

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
device

'cuda'

Separacion de dataset en entrenamiento, validacion y prueba.

In [6]:
train_dataset, val_dataset, test_dataset = WikiText2()

Se importa el diccionario de los tokens en ingles.

In [7]:
tokeniser = get_tokenizer('basic_english')
def yield_tokens(data):
    for text in data:
        yield tokeniser(text)

Del data set de entrenamiento se define el vocabulario considerando la omisión de caracteres especiales.

In [8]:
# Build the vocabulary
vocab = build_vocab_from_iterator(yield_tokens(train_dataset), specials=["<unk>", "<pad>", "<bos>", "<eos>"])
#set unknown token at position 0
vocab.set_default_index(vocab["<unk>"])

Con el dataset se preprocesan los datos de texto para el modelo de lenguaje. La función data_process toma una lista de cadenas de texto sin procesar y una longitud de secuencia como entrada. Luego, tokeniza el texto y convierte los tokens en tensores. Finalmente, elimina los tensores vacíos y crea tensores para los conjuntos de entrenamiento, validación y prueba.

In [9]:
seq_length = 50
def data_process(raw_text_iter, seq_length = 50):
  data = [torch.tensor(vocab(tokeniser(item)), dtype=torch.long) for item in raw_text_iter]
  data = torch.cat(tuple(filter(lambda t: t.numel() > 0, data))) #remove empty tensors
  return (data[:-(data.size(0)%seq_length)].view(-1, seq_length),
          data[1:-(data.size(0)%seq_length-1)].view(-1, seq_length))
# # Create tensors for the training set
x_train, y_train = data_process(train_dataset, seq_length)
x_val, y_val = data_process(val_dataset, seq_length)
x_test, y_test = data_process(test_dataset, seq_length)

In [10]:
x_train.shape

torch.Size([40999, 50])

In [11]:
y_train.shape

torch.Size([40999, 50])

Transformacion a forma de Tensor para los modelos.

In [13]:
train_dataset = TensorDataset(x_train, y_train)
val_dataset = TensorDataset(x_val, y_val)
test_dataset = TensorDataset(x_test, y_test)

Se utilizan Data loades para un eficiente carga de la informacion tambien sirve para iterar a traves de los batch para el entrenamiento.

In [15]:
batch_size = 64  # choose a batch size that fits your computation resources
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, drop_last=True)

Definimos el modelo LSTM que esta basado en una red neuronal completamente conectada. Este modelo va a predecir la siguente palabra en la secuencia.



**def forward(self, text, hidden)**: Este método define el pase adelante del modelo. Toma dos argumentos:
  text: Un tensor de índices de palabras que representa la secuencia de entrada.
  hidden: Una tupla de tensores de estado oculto que representa el estado oculto inicial de la capa LSTM.

**embeddings = self.embeddings(text):** Esta línea aplica la capa de incrustación a la secuencia de entrada, convirtiendo cada índice de palabra en una representación vectorial.

**output, hidden = self.lstm(embeddings, hidden):** Esta línea aplica la capa LSTM a la secuencia de entrada incrustada, produciendo un tensor de salida y actualizando el estado oculto.

**decoded = self.fc(output):** Esta línea aplica la capa lineal totalmente conectada a la salida de la capa LSTM, produciendo un tensor de logaritmos sobre el vocabulario.

**def init_hidden(self, batch_size):** Este método inicializa el estado oculto para la capa LSTM. Toma un argumento:
  batch_size El tamaño del lote.

**return (torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device),:** Esta línea crea un tensor cero para el estado oculto de cada capa en el lote y lo convierte al dispositivo especificado (por ejemplo, CPU o GPU).

**torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)):** Esta línea crea un tensor cero para el estado de la celda de cada capa en el lote y lo convierte al dispositivo especificado.


In [14]:
# Define the LSTM model
# Feel free to experiment
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
        super(LSTMModel, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embed_size)
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, text, hidden):
        embeddings = self.embeddings(text)
        output, hidden = self.lstm(embeddings, hidden)
        decoded = self.fc(output)
        return decoded, hidden

    def init_hidden(self, batch_size):

        return (torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device),
                torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device))



vocab_size = len(vocab) # vocabulary size
emb_size = 100 # embedding size
neurons = 128 # the dimension of the feedforward network model, i.e. # of neurons
num_layers = 1 # the number of nn.LSTM layers
model = LSTMModel(vocab_size, emb_size, neurons, num_layers)


Con el modelo definido se procede a estructurar la fase de entrenamiento, considerando las epocas y el optimizador.

In [16]:
def train(model, epochs, optimiser):
  model = model.to(device=device)                                         # loads model into the given device
  model.train()                                                           # sets the model in training model

  for epoch in range(epochs):
    (h, c) = model.init_hidden(batch_size)                                # initialize hidden state
    for i, (data, targets) in enumerate((train_loader)):
      h = h.detach()                                                      # detaches tensor from current graph (result will no require gradient)
      c = c.detach()

      inputs, targets = data.to(device), targets.to(device)               # loads x and y into the given device
      outputs, (h, c) = model(inputs, (h, c))                             # performs forward pass

      loss = loss_function(outputs[: , :, -1], targets.float())           # computes loss
      optimiser.zero_grad()                                               # resets the gradients of all optimized tensors
      loss.backward()                                                     # performs backward pass
      torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)
      optimiser.step()                                                    # updates parameters
      if (i+1) % 100 == 0:
        print (f'Epoch [{epoch+1}/{epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')
  print("Training complete")

Con el modelo configurado se procede al entrenamiento

In [22]:
# Call the train function
loss_function = nn.CrossEntropyLoss().to(device)  #Esta línea define la función de pérdida utilizada para evaluar el rendimiento del modelo durante el entrenamiento. La función de pérdida utilizada es nn.CrossEntropyLoss, que es una función de pérdida común para tareas de clasificación.
lr = 0.0005                                       #establece la tasa de aprendizaje, que es un parámetro que controla la magnitud de las actualizaciones de parámetros durante el entrenamiento. Una tasa de aprendizaje más alta puede conducir a un aprendizaje más rápido, pero también puede aumentar la inestabilidad y el riesgo de sobreajuste.
epochs = 5                                        #establece el número de épocas, que es el número de veces que se recorrerá todo el conjunto de datos de entrenamiento durante el proceso de aprendizaje. Un mayor número de épocas puede conducir a un mejor rendimiento, pero también puede aumentar el tiempo de entrenamiento.
optimiser = optim.Adam(model.parameters(), lr=lr) #crea el optimizador, que es un algoritmo utilizado para actualizar los parámetros del modelo durante el entrenamiento. El optimizador utilizado es optim.Adam, que es un optimizador común para redes neuronales.
train(model, epochs, optimiser)                   #llama a la función train para entrenar el modelo. La función train toma el modelo, el número de épocas y el optimizador como parámetros, y entrena el modelo utilizando el conjunto de datos de entrenamiento y la función de pérdida definida anteriormente.

Epoch [1/5], Step [100/640], Loss: 368257.7500
Epoch [1/5], Step [200/640], Loss: 313254.0000
Epoch [1/5], Step [300/640], Loss: 353371.7500
Epoch [1/5], Step [400/640], Loss: 323731.2188
Epoch [1/5], Step [500/640], Loss: 309512.5938
Epoch [1/5], Step [600/640], Loss: 330767.4375
Epoch [2/5], Step [100/640], Loss: 336133.1562
Epoch [2/5], Step [200/640], Loss: 323307.1875
Epoch [2/5], Step [300/640], Loss: 322032.9375
Epoch [2/5], Step [400/640], Loss: 322483.7812
Epoch [2/5], Step [500/640], Loss: 301092.5000
Epoch [2/5], Step [600/640], Loss: 344871.7500
Epoch [3/5], Step [100/640], Loss: 333856.3125
Epoch [3/5], Step [200/640], Loss: 345922.8438
Epoch [3/5], Step [300/640], Loss: 338726.3438
Epoch [3/5], Step [400/640], Loss: 332800.4375
Epoch [3/5], Step [500/640], Loss: 385547.9375
Epoch [3/5], Step [600/640], Loss: 343711.1562
Epoch [4/5], Step [100/640], Loss: 337052.7812
Epoch [4/5], Step [200/640], Loss: 348363.6875
Epoch [4/5], Step [300/640], Loss: 295095.2500
Epoch [4/5], 

Evaluacion del modelo generativo, como secuencia inicial se manda la frase "I like" y se restringe a 100 palabras totales.

In [19]:
def generate_text(model, start_text, num_words, temperature=1.0):

    model.eval()                            #Pone el modelo en modo de evaluación, lo que desactiva ciertas optimizaciones que solo son aplicables durante el entrenamiento.
    words = tokeniser(start_text)           #Tokeniza el texto inicial utilizando el tokenizador proporcionado, dividiendo el texto en una lista de palabras individuales.
    hidden = model.init_hidden(1)           #inicializa el estado oculto de la capa LSTM, que se utiliza para rastrear la memoria interna del modelo durante la generación de texto.
    for i in range(0, num_words):
        x = torch.tensor([[vocab[word] for word in words[i:]]], dtype=torch.long, device=device)  # crea un tensor que representa la secuencia de entrada para la iteración actual. El tensor selecciona las palabras de words comenzando en el índice i y las convierte a sus índices correspondientes en el vocabulario.
        y_pred, hidden = model(x, hidden)                                                         #pasa el tensor de secuencia de entrada x y el estado oculto actual hidden al modelo. Devuelve los logaritmos de salida previstos y_pred y actualiza el estado oculto hidden.
        last_word_logits = y_pred[0][-1]                                                          #extrae los logaritmos para la última palabra en la secuencia de salida prevista.
        p = (F.softmax(last_word_logits / temperature, dim=0).detach()).to(device='cpu').numpy()  #aplica la activación softmax a los logaritmos, los convierte a probabilidades y transfiere la distribución de probabilidad resultante a la CPU.
        word_index = np.random.choice(len(last_word_logits), p=p)                                 #selecciona un índice de palabra aleatorio de la distribución de probabilidad p.
        words.append(vocab.lookup_token(word_index))                                              #agrega la palabra seleccionada a la lista de palabras generadas.

    return ' '.join(words)

In [21]:
#Genera 60 palabras a partir del texto inicial "I like".
print(generate_text(model, start_text="I like", num_words=60))

i like blossom kennedy harriet drumming 65th asserts corridors perón 1966 swim mausoleum splashes wading professional bodies convicts source molecule viola sakuraba jill beer colfer jfk seismic enforce widows crevice captained boletus enabling ethnic hinting larry among kelp morphology parsons monsen westbound mancuso venture factions suitability 1660 1207 seed interrupts brightly cadre dunn statute sachs kalyanasundara egg popularizing assumption imjin tennant fiancé


# Conclusiones

Los modelos generativos son un tipo de modelo de aprendizaje automático que se utilizan para generar nuevas muestras de datos. Estos modelos son útiles para una variedad de tareas, como la creación de contenido creativo, la traducción de idiomas y la detección de fraudes.

En el código anterior, se muestra cómo utilizar un modelo generativo LSTM para generar texto. El modelo se entrena en un conjunto de datos de texto, y luego se puede utilizar para generar nuevo texto. La función generate_text() del código toma un texto inicial como entrada y genera un número especificado de palabras.

El código también muestra cómo entrenar un modelo generativo. La función train() del código toma el modelo, el número de épocas y el optimizador como parámetros. La función recorre el conjunto de datos de entrenamiento varias veces, actualizando los parámetros del modelo cada vez.

Los modelos generativos tienen el potencial de revolucionar una variedad de industrias. En el ámbito del marketing, los modelos generativos se pueden utilizar para crear anuncios personalizados o contenido creativo. En el ámbito de la educación, los modelos generativos se pueden utilizar para crear materiales de aprendizaje personalizados o para ayudar a los estudiantes a escribir ensayos. En el ámbito de la atención médica, los modelos generativos se pueden utilizar para ayudar a los médicos a diagnosticar enfermedades o para desarrollar nuevos medicamentos.

Sin embargo, los modelos generativos también presentan algunos desafíos. Un desafío es que los modelos generativos pueden generar texto que es sesgado o inexacto. Otro desafío es que los modelos generativos pueden ser utilizados para crear contenido dañino, como noticias falsas o propaganda.

A pesar de estos desafíos, los modelos generativos tienen el potencial de ser una herramienta poderosa para el bien. Con el desarrollo de nuevas técnicas y la atención a los desafíos, los modelos generativos tienen el potencial de mejorar nuestras vidas de muchas maneras.

Algunos ejemplos específicos de cómo se están utilizando los modelos generativos en la actualidad incluyen:

En el ámbito de la educación, los modelos generativos se utilizan para ayudar a los estudiantes a aprender. Por ejemplo, la compañía Pearson utiliza un modelo generativo para crear ejercicios de práctica personalizados para los estudiantes.

Estos son solo algunos ejemplos de cómo se están utilizando los modelos generativos en la actualidad. A medida que los modelos generativos continúen desarrollándose, es probable que veamos aún más casos de uso innovadores para esta tecnología.